# LMIs and Semidefinite Programming

A semidefinite program (SDP) is an optimization problem with linear objective and LMI and linear equality constraints:  

$$\text{Minimize }\ c^Tx$$
$$\text{subject to }\ F_0+x_1F_1+\cdot\cdot\cdot+x_nF_n\ge0$$
$$Ax=b$$

In [22]:
import numpy as np
import cvxpy as cp
import cvxopt as co

**Problem:** A university has decided to construct a grassy (elliptical) oval in an area bounded by existing buildings. Unfortunately, the area is not itself elliptical. The available land area is bounded by a quadrilateral with vertices (0,0), (0,100), (150,150) and (300,0), where all distances are in metres. Determine the dimensions of the largest elliptical garden that will fit in the space bounded by these vertices, in terms of its center and its semi-major and semi-minor axes.

In [1]:
from cvxopt import matrix, solvers


# Objective function: c.T * x
# Subject to: F0 + x1.F1 + .. + xn.Fn
# Equality constraint: Ax + b

# The arrays that limit the elliptical shape after normalizing by 100
edges = np.array([[-1/3, 1], [1,1], [0,1], [1,0]])
points = np.array([[1], [0], [0], [3]])

# Variable((n, n), PSD=True) instead of Semidef(n)
Q = cp.Variable((n,n), PSD=True)

constraints += [Q * edges[i] - points[i] >= 0 for i in range(4)]

obj = cvx.Maximize(cvx.log_det(Q))

cvx.Problem(obj, constraints)